In [1]:
import json
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime as dt
import time

In [65]:
def get_poem_list(num):
    r = requests.get("https://www.poets.org/poetsorg/poem-day?utm_=&page={}".format(num))
    page = r.text
    soup = BeautifulSoup(page, 'html.parser')
    if soup.tbody == None:
        return {}
    else:
        poems = [{'title': x.get_text(), 'url':x['href']} for x in soup.tbody.find_all('a') if 'poem' in x['href']]
        authors = [{'author':x.get_text(), 'author_url': x['href']} for x in soup.tbody.find_all('a') if 'poet/' in x['href']]
        dates = [{'date': x.get_text()} for x in soup.tbody.find_all('span')]
        output = [{**poems[p], **authors[p], **dates[p], **{'page_list':num}} for p in range(len(poems))]
        return {'poems':output}

In [68]:
poem_list = get_poem_list(10)

In [53]:
pd.DataFrame(poem_list)

,poems
0,{'title': 'Poem Begun on the Day of My Father’...
1,{'title': 'I Don't Know What Will Kill Us Firs...
2,"{'title': 'Adjacent, Against, Upon', 'url': '/..."
3,"{'title': 'Phlebotomy, as Told by the Blood', ..."
4,"{'title': 'Trees', 'url': '/poetsorg/poem/tree..."
5,{'title': 'Prompts (for High School Teachers W...
6,{'title': 'Magpies Recognize Themselves in the...
7,"{'title': 'Ultrasound', 'url': '/poetsorg/poem..."
8,"{'title': 'A Skull', 'url': '/poetsorg/poem/sk..."
9,"{'title': 'Miscegenation', 'url': '/poetsorg/p..."


In [175]:
def get_related_title(text):
    if ' by ' in text:
        split = text.split(' by ')
        return split
    else:
        return [text, '']
    
def get_poem(path):
    r = requests.get("https://www.poets.org/{}".format(path))
    page = r.text
    soup = BeautifulSoup(page, 'html.parser')

    # poem text
    try:
        poem = soup.find('pre').get_text()
    except:
        try:
            t = [str(p) for p in soup.find('div', {'class':'field field-name-body field-type-text-with-summary field-label-hidden'}).find_all('p')]
            poem = ''.join(t)
        except:    
            poem = ''
            print("Issue getting poem on {}".format(path))

    # author bio
    try:
        bio = soup.find('div', {'class':'views-field views-field-path'}).find('p').get_text()
    except:
        try:
            bio = soup.find('div', {'class':'views-field views-field-path'}).get_text()
        except:
            bio = ''
            print("Issue getting bio on {}".format(path))
    
    # related poems
    related_soup = soup.find('div', {"id": "block-aap-related-content-related-poems"})
    if related_soup == None:
        related = []
    else:
        related_soup_list = related_soup.find_all('li') # related 
        if len(related_soup_list) > 0:
            related_poems = [{'title': get_related_title(x.get_text()), 'author': get_related_title(x.get_text())} for x in related_soup_list]
            related_urls = [{'url': x.find('a')['href']} for x in related_soup_list]
            related = [{**related_poems[n], **related_urls[n]} for n in range(len(related_poems))]
        else:
            related = []
        
    # occasions
    occasion_soup = soup.find('div', {"id": "block-aap-poems-poems-occasions"})
    if occasion_soup == None:
        occasions = []
    else:
        occasions = [{'id': x['href'].replace('/poetsorg/poems?field_occasion_tid=',''), 'occasion': x.get_text()} for x in occasion_soup.find_all('a')]

    # forms
    form_soup = soup.find('div', {"id": "block-aap-poems-poems-forms"})
    if form_soup == None:
        forms = []
    else: 
        forms = [{'form_id': x['href'].replace('/poetsorg/poems?field_form_tid=',''), 'form': x.get_text()} for x in form_soup.find_all('a')]
        
    # themes, schools, will need to find join a list ??
    return {'text': poem, 'author_bio': bio, 'related_poems': related, 'occasions': occasions, 'forms': forms}

In [34]:
def get_list_with_poems(poem_list):
    urls = [p['url'] for p in poem_list['poems']]
    poems = [get_poem(url) for url in urls]
    output = [{**poem_list['poems'][n], **poems[n]} for n in range(len(urls))]
    return output

In [ ]:
def get_max_page():
    r = requests.get("https://www.poets.org/poetsorg/poem-day?utm_=&page0")
    page = r.text
    soup = BeautifulSoup(page, 'html.parser')
    max_page = soup.find('li',{'class':{'pager-last last'}}).get_text()
    return int(max_page)

In [35]:
poems = get_list_with_poems(poem_list)

In [149]:
def get_all_poems(start_page = 0, end_page = 2000):
    all_poems = []
    n = start_page
    while n <= end_page: # max
        print(n)
        #if (n % 10) == 0 :
        #    print("On page {}".format(n))
        poem_list = get_poem_list(n)
        if poem_list == {}:
            print("No more poems at page {}".format(n))
            break
        else:
            poems = get_list_with_poems(poem_list)
            all_poems += poems
        n += 1
    return all_poems

In [176]:
all_poems = get_all_poems(65, end_page = get_max_page())

65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323

In [179]:
first_poems = get_all_poems(0, 64)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [177]:
len(all_poems)

2772

In [180]:
len(first_poems)

650

In [181]:
final_poems = first_poems + all_poems
len(final_poems)

3422

In [80]:
get_max_page()

343

In [183]:
def poems_to_json(poems):
    with open('poems.json', 'w') as fp:
        json.dump(poems, fp)

In [184]:
poems_to_json(final_poems)

In [185]:
pd.DataFrame(final_poems)

,author,author_bio,author_url,date,forms,occasions,page_list,related_poems,text,title,url
0,Dean Rader,Dean Rader is the author of Self-Portrait as a...,/poetsorg/poet/dean-rader,"May 09, 2019",[],"[{'id': '492', 'occasion': 'Funerals'}]",0,[{'title': ['René Descartes and the Clockwork ...,Light the last light and lift—\n ...,Poem Begun on the Day of My Father’s Funeral a...,/poetsorg/poem/poem-begun-day-my-fathers-funer...
1,Fatimah Asghar,Fatimah Asghar is the author of the full-lengt...,/poetsorg/poet/fatimah-asghar,"May 08, 2019",[],"[{'id': '1556', 'occasion': 'Earth Day'}]",0,"[{'title': ['Everglades', 'Anne Marie Macari']...",so I count my hopes: the bumblebees \nare maki...,I Don't Know What Will Kill Us First: The Race...,/poetsorg/poem/i-dont-know-what-will-kill-us-f...
2,Rick Barot,Rick Barot was born in the Philippines in 1969...,/poetsorg/poet/rick-barot,"May 07, 2019",[],"[{'id': '1806', 'occasion': 'LGBTQ Pride Month'}]",0,[{'title': ['Prompts (for High School Teachers...,—after Michael Heizer\n\nI may be looking at t...,"Adjacent, Against, Upon",/poetsorg/poem/adjacent-against-upon
3,torrin a. greathouse,torrin a. greathouse is the author of two chap...,/poetsorg/poet/torrin-greathouse,"May 06, 2019",[],[],0,"[{'title': ['Nearly', 'Idra Novey'], 'author':...",Consider these parallel histories: An emperor ...,"Phlebotomy, as Told by the Blood",/poetsorg/poem/phlebotomy-told-blood
4,Joyce Kilmer,"Joyce Kilmer was born on December 6, 1886, in ...",/poetsorg/poet/joyce-kilmer,"May 05, 2019",[],"[{'id': '495', 'occasion': 'Summer'}]",0,"[{'title': ['Song of Myself, 25', 'Walt Whitma...",I think that I shall never see \nA poem love...,Trees,/poetsorg/poem/trees
5,Dante Di Stefano,Dante Di Stefano is the winner of the 2019 On ...,/poetsorg/poet/dante-di-stefano,"May 04, 2019",[],[],0,"[{'title': ['Edgar Allan Poe', 'Timothy Thomas...",Write about walking into the building\nas a ne...,Prompts (for High School Teachers Who Write Po...,/poetsorg/poem/prompts-high-school-teachers-wh...
6,Kelli Russell Agodon,Kelli Russell Agodon is the author of Hourglas...,/poetsorg/poet/kelli-russell-agodon,"May 03, 2019",[],[],0,"[{'title': ['Words', 'John Keene'], 'author': ...",The night sounds like a murder \nof magpies an...,Magpies Recognize Themselves in the Mirror,/poetsorg/poem/magpies-recognize-themselves-mi...
7,Kaveh Akbar,Kaveh Akbar is the author of Calling a Wolf a ...,/poetsorg/poet/kaveh-akbar,"May 02, 2019",[],[],0,"[{'title': ['Nearly', 'Idra Novey'], 'author':...",my father is tying concertina wire\naround his...,Ultrasound,/poetsorg/poem/ultrasound
8,Dana Levin,Dana Levin is the author of Banana Palace (Cop...,/poetsorg/poet/dana-levin,"May 01, 2019",[],[],0,"[{'title': ['Roof Nightclub', 'Ishion Hutchins...",is like a house\n with a brain insi...,A Skull,/poetsorg/poem/skull
9,Nick Laird,"Nick Laird is a poet, novelist, screenwriter, ...",/poetsorg/poet/nick-laird,"April 30, 2019",[],[],0,"[{'title': ['Thinking of Frost', 'Major Jackso...",Even this freckle testifies to the strength \n...,Miscegenation,/poetsorg/poem/miscegenation
